In [1]:
#文件说明
# train.csv, test.csv
# row_id: 签到事件的id
# x, y: 坐标
# time: 时间戳
# place_id: 签到的位置，标签

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

C:\Users\17634\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\17634\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\17634\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\17634\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
facebook = pd.read_csv(r"D:\程序语言\人工智能教程\阶段3-人工智能机器学习\02_机器学习算法day06\02-代码\data\FBlocation\train.csv")
facebook

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949
...,...,...,...,...,...,...
29118016,29118016,6.5133,1.1435,67,399740,8671361106
29118017,29118017,5.9186,4.4134,67,125480,9077887898
29118018,29118018,2.9993,6.3680,67,737758,2838334300
29118019,29118019,4.0637,8.0061,70,764975,1007355847


In [4]:
facebook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
row_id      int64
x           float64
y           float64
accuracy    int64
time        int64
place_id    int64
dtypes: float64(2), int64(4)
memory usage: 1.3 GB


In [5]:
#缩小数据范围
facebook_data = facebook.query('x>2 & x<2.5 & y>2 & y<2.5')
facebook_data

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909
...,...,...,...,...,...,...
29116142,29116142,2.0804,2.0657,168,217886,1247398579
29116267,29116267,2.4309,2.4646,33,314360,1951613663
29116295,29116295,2.1797,2.1707,89,74954,4724115005
29116475,29116475,2.3924,2.2704,62,206257,2819110495


In [8]:
facebook_data.groupby('place_id').count()

,row_id,x,y,accuracy,time
place_id,,,,,
1006234733,1,1,1,1,1
1008823061,4,4,4,4,4
1012580558,3,3,3,3,3
1025585791,21,21,21,21,21
1026507711,220,220,220,220,220
...,...,...,...,...,...
9986101718,1,1,1,1,1
9993141712,1,1,1,1,1
9995108787,23,23,23,23,23


In [9]:
time = pd.to_datetime(facebook_data.time, unit='s')
# 选择时间特征
time = pd.DatetimeIndex(time)
facebook_data['day'] = time.day
facebook_data['hour'] = time.hour
facebook_data['weekday'] = time.weekday
# 去掉签到较少的地方
place_count = facebook_data.groupby('place_id').count()
place_count = place_count[place_count.row_id > 3]
facebook_data = facebook_data[facebook_data["place_id"].isin(place_count.index)]
# 2.4 确定特征值和目标值
x = facebook_data[["x", "y", "accuracy", "day", "hour", "weekday"]]
y = facebook_data["place_id"]
x

C:\Users\17634\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\17634\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\17634\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,x,y,accuracy,day,hour,weekday
163,2.1663,2.3755,84,8,18,3
310,2.3695,2.2034,3,3,17,5
658,2.3236,2.1768,66,6,19,1
1368,2.2613,2.3392,73,4,16,6
1627,2.3331,2.0011,66,7,21,2
...,...,...,...,...,...,...
29116142,2.0804,2.0657,168,3,12,5
29116267,2.4309,2.4646,33,4,15,6
29116295,2.1797,2.1707,89,1,20,3
29116475,2.3924,2.2704,62,3,9,5


In [10]:
y


163         3869813743
310         2636621520
658         7877745055
1368        9775192577
1627        6731326909
               ...    
29116142    1247398579
29116267    1951613663
29116295    4724115005
29116475    2819110495
29117203    2634419689
Name: place_id, Length: 69264, dtype: int64

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [12]:
estimator = KNeighborsClassifier()
param_grid = {"n_neighbors": [1,3,5,7,9]}
estimator = GridSearchCV(estimator, param_grid=param_grid, cv=5)
estimator.fit(x_train, y_train)

C:\Users\17634\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': [1, 3, 5, 7, 9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [13]:
score = estimator.score(x_test, y_test)
print("最后预测的准确率为：\n", score)

print("交叉验证的最好结果：\n", estimator.best_score_)
print("最好的数据模型：\n", estimator.best_estimator_)

最后预测的准确率为：
 0.36515361515361516
交叉验证的最好结果：
 0.35462385462385465
最好的数据模型：
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')
